# MFCC - Pré-Processamento de Sinais de Áudio

Neste notebook é possível selecionar qualquer arquivo de áudio (em formato .wav) e gerar a MFCC deste sinal.

Utilizando a biblioteca Librosa são exibidos os gráficos da MFCC, seus Deltas, o espectrograma do sinal e uma reconstrução a partir dos coeficientes da MFCC. Alguns arquivos de áudio já estão inclusos, porém você pode utilizar um arquivo próprio, basta colocá-lo na pasta "audio", que está localizada no mesmo diretório deste notebook.

Caso deseje pular para alguma parte em específico basta clicar nos links abaixo:

[Bibliotecas](#Importando-as-Bibliotecas) | [Funções Utilizadas](#Funções) | [Gráficos das MFCCs](#Exibição-dos-expectogramas) | [Reconstrução do Sinal](#Reconstrução-do-sinal-à-partir-dos-coeficientes) 

# Participantes do projeto
- Gabriel Santos Gollo do Amaral
- Marcial Faria Vasconcelos

## Instalação de Bibliotecas

Descomente as linhas abaixo caso seja necessário instalar alguma das bibliotecas.

In [1]:
# !pip install numpy
# !pip install control
# !pip install scipy
# !pip install IPython
# !pip install librosa

## Importando as Bibliotecas

Foram utilizados recursos das seguintes bibliotecas:

- [numpy](https://numpy.org/doc/stable/user/whatisnumpy.html)
- [scipy](https://docs.scipy.org/doc/scipy/reference/)
- [librosa](https://librosa.org/doc/latest/index.html)
- [matplotlib](https://matplotlib.org/stable/api/index)
- [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/)
- [os](https://docs.python.org/3/library/os.html)
- [IPython](https://ipython.org/documentation.html)

In [2]:
import numpy as np, scipy as sp
import matplotlib.pyplot as plt, IPython.display as ipd
import librosa, librosa.display
import ipywidgets as widgets
import os
import warnings
warnings.filterwarnings('ignore')
from ipywidgets import interact, interactive, fixed, interact_manual

# %matplotlib widget
%matplotlib inline

## Funções

Nesta célula foram definidas as funções utilizadas para construção dos MFCCs e dos gráficos.

In [3]:
FIGURE_X = 15
FIGURE_Y = 10
N_MFCC   = 13

def makeMFCC(x, sr):
    return librosa.feature.mfcc(x, n_mfcc=N_MFCC, sr=sr)

def plotMFCC(mfcc):
    plt.figure(figsize=(FIGURE_X, FIGURE_Y))
    librosa.display.specshow(mfcc, x_axis='time', y_axis='frames')
    plt.colorbar()
    plt.title('MFCC')
    plt.tight_layout()
    plt.show()

def plotSpectrogram(x, sr):
    plt.figure(figsize=(FIGURE_X, FIGURE_Y))
    librosa.display.specshow(librosa.amplitude_to_db(librosa.stft(x), ref=np.max), y_axis='log', x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Power spectrogram')
    plt.tight_layout()
    plt.show()

def plotOriginalSignal(x):
    librosa.load(x)
    plt.figure(figsize=(FIGURE_X, FIGURE_Y))
    librosa.display.waveplot(x)
    plt.title('Original Signal')
    plt.tight_layout()
    plt.show()
    
def showSound(audio,sr=22050,title="Sinal de audio",xlabel="tempo (s)",ylabel="Amplitude"):
    N = len(audio)
    T = N/sr
    t = np.linspace(0,T,N)
    plt.figure(figsize=(7, 2.5))
    plt.plot(t,audio)
    plt.title(title)
    plt.xlabel(xlabel);   plt.ylabel(ylabel);
    plt.grid(which='both')
    plt.show()
    ipd.display(ipd.Audio(audio, rate=sr, autoplay=False))

def getDelta(mfcc):
    return librosa.feature.delta(mfcc)

def getDeltaDelta(mfcc):
    return librosa.feature.delta(mfcc, order=2)

def displayMfccAndDeltas(mfcc, delta, deltaDelta):
    plt.figure(figsize=(FIGURE_X, FIGURE_Y))
    plt.subplot(3, 1, 1)
    librosa.display.specshow(mfcc, x_axis='time', y_axis='frames')
    plt.colorbar(format="%+.1f")
    plt.title('MFCC', fontsize=18)
    plt.xlabel('tempo (s)', fontsize=15);plt.ylabel('Coeficientes', fontsize=15);
    plt.subplot(3, 1, 2)
    librosa.display.specshow(delta, x_axis='time', y_axis='frames')
    plt.colorbar(format="%+.1f")
    plt.title('MFCC Delta', fontsize=18)
    plt.xlabel('tempo (s)', fontsize=15);plt.ylabel('Coeficientes', fontsize=15);
    plt.subplot(3, 1, 3)
    librosa.display.specshow(deltaDelta, x_axis='time', y_axis='frames')
    plt.colorbar(format="%+.1f")
    plt.title('MFCC Delta Delta', fontsize=18)
    plt.xlabel('tempo (s)', fontsize=15);plt.ylabel('Coeficientes', fontsize=15);
    plt.tight_layout()
    plt.show()

def plotFFT(x, sr):
    plt.figure(figsize=(FIGURE_X, FIGURE_Y/2))
    librosa.display.specshow(librosa.amplitude_to_db(librosa.stft(x), ref=np.max), y_axis='log', x_axis='time')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.tight_layout()
    plt.show()

def processAudio(x, sr):
    mfcc = makeMFCC(x, sr)
    delta = getDelta(mfcc)
    deltaDelta = getDeltaDelta(mfcc)
    plotFFT(x, sr)
    print("MFCC:", mfcc.shape)
    displayMfccAndDeltas(mfcc, delta, deltaDelta)

def startProcess(path):
    x, sr = librosa.load(path)
    showSound(x, sr, title="")
    processAudio(x, sr)
    
def reverseMFCC(coefficients):
    return librosa.feature.inverse.mfcc_to_audio(coefficients)
    
def reconstruct(path):
    x, sr = librosa.load(path)
    showSound(x, sr, title="Sinal Original")
    mfcc = makeMFCC(x, sr)
    xr   = reverseMFCC(mfcc)
    showSound(xr, sr, title="Sinal Reconstruído")

## Cálculo dos MFCCs 

### Carregar a lista de arquivos

In [4]:
fileList = []
pasta = "./audio"
for diretorio, subpastas, arquivos in os.walk(pasta):
    for arquivo in arquivos:
        fileList.append(( arquivo.rsplit( ".", 1 )[ 0 ] , pasta+"/"+arquivo))
# print(files)

### Exibição dos expectogramas

Selecione na caixa de diálogo abaixo o arquivo de áudio que deseja analisar: 

In [5]:
interact(startProcess, path=widgets.Dropdown(options = fileList, 
                                             description='Select a file:', #value=None,
                                             disabled=False), continuous_update=False);


interactive(children=(Dropdown(description='Select a file:', options=(('acusticguitar', './audio/acusticguitar…


## Reconstrução do **sinal** à partir dos coeficientes
---
`Obs.: Áudios longos podem demorar um certo tempo para serem reconstruidos `

In [6]:
interact(reconstruct, path=widgets.Dropdown(options= fileList, 
                                            description='Select a file:', #value=None,
                                            disabled=False), continuous_update=False);

interactive(children=(Dropdown(description='Select a file:', options=(('acusticguitar', './audio/acusticguitar…